This notebook contains the detailed cross-correlation extraction and analysis.

- **Author**: Benkirane Ismail
- **Email**: [ibenkirane@mgb.org](mailto:ibenkirane@mgb.org)
- **Version**: 1.0.0
- **Date**: 2023-10-19


## Imports

In [ ]:
import sys
import pandas as pd

sys.path.append('../')

from utils import UTILITIES, CORRELATION, CrossCorrelation

## Get Metadata

In [ ]:
desired_measurement = ['Empatica', 'Audio', 'FaceReader', 'GoPro']
save = True

groups = {
    'Cluster 1' : [1003, 1007, 1013, 1015, 1020, 1024, 1026],
    'Cluster 2' : [1001, 1031, 1032, 1037, 1039],
    'Cluster 3' : [1008, 1017, 1022, 1025, 1033, 1040, 1041, 1042],
    'All Subjects': [1001, 1003, 1007, 1008, 1013, 1015, 1017, 1020, 1022, 1024, 1025, 1026, 1031, 1032, 1033, 1037, 1039, 1040, 1041]
    }

In [ ]:
utilities = UTILITIES()
correlation = CORRELATION(desired_measurement)
cross_correlation = CrossCorrelation(desired_measurement)

# Express Cross-Correlation extraction

In [ ]:
if save:
    cross_correlation.save_all_cross_correlation_results(desired_measurement=desired_measurement, subject_groups = groups, include_pairs = True, only_pairs=False)
    #cross_correlation.save_all_shift_stat_graphs(desired_measurement=['Empatica', 'Audio', 'Webcam', 'GoPro'])

# Data Loading

In [ ]:
if isinstance(desired_measurement, str):
    all_features = pd.read_csv(f'../computed_features/{desired_measurement}/all_features_windows.csv')
    stand_features = pd.read_csv(f'../computed_features/{desired_measurement}/stand_features_windows.csv')
else:
    all_features = pd.read_csv(f'../computed_features/all_features_windows.csv')
    stand_features = pd.read_csv(f'../computed_features/stand_features_windows.csv')

features_names = utilities.get_feature_names(all_features, desired_measurement)

features_to_cross_correlate = cross_correlation.get_features_to_cross_correlate(features_names)

In [ ]:
groups = {
    'g0': [1001, 1007],
    'g1': [1003, 1008, 1013, 1015, 1020, 1022, 1024, 1025, 1026, 1033, 1040, 1041], 
    'g2': [1031, 1032, 1037, 1039, 1042]
}

features_to_use = utilities.select_features_from_subjects(stand_features, groups['g1'], include_neutral=True)

# Grouping the features per emotion

In [ ]:
features_grouping = utilities.group_features_by_label(features_to_use, windows=True)

In [ ]:
for emotion in features_grouping.keys():
    for row in range(len(features_grouping[emotion])):
        features = features_grouping[emotion].iloc[row].index
        for feature in features:
            if feature != 'label':
                basic = len(features_grouping[emotion].iloc[row][features[1]])
                if basic != len(features_grouping[emotion].iloc[row][feature]):
                    print(emotion, row, feature, basic, len(features_grouping[emotion].iloc[row][feature]))

## Cross Correlation Analysis

In [ ]:
cross_correlations = cross_correlation.get_cross_correlations(features_grouping, features_to_cross_correlate)
lags_dict = cross_correlation.compare_max_lags(cross_correlations, features_names, intra_comparison = False, save = save)
shifted_feature_pairs, centered_feature_pairs = cross_correlation.get_lag_stats(lags_dict, features_names, p_value = 0.05, verbose = False, save = save) 

Positive Lag: f1 leads f2

Negative Lag: f2 leads f1

In [ ]:
positive_shifts, negative_shifts, no_shifts = cross_correlation.get_shifts_stats(shifted_feature_pairs, features_names, save = save)

In [ ]:
if save:
    cross_correlation.save_cross_corr_graphs_to_pdf(cross_correlations, features_names)

## Results

In [ ]:
import json

with open ('Analysis/Cross-Correlation/All Subjects/Audio/shifted_feature_pairs.json', 'r') as f:
    shifted_feature_pairs = json.load(f)

In [ ]:
consistent_shift_pos, consistent_shift_neg = cross_correlation.get_consistent_shift_pairs(shifted_feature_pairs, verbose = True)

In [ ]:
cross_correlation.plot_cross_correlation(cross_correlations, 1001, 'Pride', 'eda_mean-accX_mean')

In [ ]:
cross_correlation.plot_shift_stats(positive_shifts, negative_shifts, no_shifts, desired_measurement)